In [40]:
import numpy as np
import twitter, re, datetime, pandas as pd
twitter_keys = {
    'consumer_key':        'L4sziHBqV4VUIfKezbos0JMVl',
    'consumer_secret':     'lJau6R7GIHFwoGR5wB3PlLQPXBChwzJFJ9WGXXtazcDSA1Vb1X',
    'access_token_key':    '941359629606539264-05XcmQfdwMXTbPNWS3r7cZThvbQBxCK',
    'access_token_secret': 'VdE3VJVk6oxbohQGcw7WYA5Tg4Sr8kW9duTO1wxmB6qXk'
}
api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret'],
    tweet_mode = 'extended'
)
print("Done")

Done


In [41]:
type(api)

twitter.api.Api

In [42]:
x = api.GetUserTimeline(screen_name="HillaryClinton", count=20, include_rts=False)
x = [_.AsDict() for _ in x]
x[0]

{'created_at': 'Thu Dec 02 19:15:17 +0000 2021',
 'favorite_count': 3245,
 'full_text': "Trump's effort to subvert our democracy is ongoing in states around the country.\n\nhttps://t.co/YBN0JRnTeb",
 'hashtags': [],
 'id': 1466486144443506698,
 'id_str': '1466486144443506698',
 'lang': 'en',
 'retweet_count': 1150,
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'urls': [{'expanded_url': 'https://www.washingtonpost.com/politics/trump-allies-election-oversight/2021/11/28/3933b3ce-4227-11ec-9ea7-3eb2406a2e24_story.html',
   'url': 'https://t.co/YBN0JRnTeb'}],
 'user': {'created_at': 'Tue Apr 09 18:04:35 +0000 2013',
  'description': '2016 Democratic Nominee, SecState, Senator, hair icon. Mom, Wife, Grandma x3, lawyer, advocate, fan of walks in the woods & standing up for our democracy.',
  'favourites_count': 1173,
  'followers_count': 31051205,
  'friends_count': 947,
  'id': 1339835893,
  'id_str': '1339835893',
  'listed_count': 42289,
  'locatio

In [43]:
for element in x:
    print(element['id'])
    print(element['full_text'])
    print('--')

1466486144443506698
Trump's effort to subvert our democracy is ongoing in states around the country.

https://t.co/YBN0JRnTeb
--
1466479975599640580
Thank you for sharing your story, Billie Jean. https://t.co/nVgsb9OK5T
--
1466461279116218370
Today is a good day to donate to Planned Parenthood and their Action Fund. https://t.co/xBMO5X5tKn

https://t.co/quAMdLZBMJ
--
1466174036166299653
I’m with @StaceyAbrams. Are you? https://t.co/RSQFVZesNM
--
1466130171996557320
Looking forward to celebrating these incredible women with all of you. https://t.co/726vYcoUSc
--
1466073531435868164
Could have been based on anyone.... https://t.co/bxYrbg1m26
--
1466054451295313920
We cannot and will not leave our granddaughters a world in which they have fewer rights and opportunities than we did.
--
1465737796019265538
In one week: 

Join me, @BenAffleck, @EricHolder, and @TraceeEllisRoss to talk about an urgent threat to our democracy—partisan gerrymandering—and what we can about it. 

https://t.co/Se8

In [44]:
y = api.GetUserTimeline(screen_name="HillaryClinton", count=20, max_id=935706980643147777, include_rts=False)
y = [_.AsDict() for _ in y]
print("Done")

Done


In [45]:
print(y[0]['id'])
print(type(y[0]))
print("\n")
for element in y:
    print(element['id'])
    print(element['full_text'])
    print('--')

933449899487780864
<class 'dict'>


933449899487780864
You go girl! This is important; costs will go up, &amp; powerful companies will get more powerful. We can’t let it slip through the cracks. https://t.co/js4agpzKeS
--
933107180131282944
And on top of that, you can’t beat a little book signing wardrobe fun.

https://t.co/K8VldCmTC6 https://t.co/9q9sTlpL3I
--
933105980484747264
And being out on the book tour it has been wonderful to hear from so many people about their activism, their courage, &amp; their resilience.
https://t.co/LhtTwV7N0d
--
933105682378870784
Wow. I wasn’t sure how letting my guard down would go...but it’s been cathartic &amp; rewarding. I loved writing this book, &amp; I’m honored to be in such great company on this list!
https://t.co/rAYpWHzxrJ
--
932746529475055618
It’s been almost 2 months since Congress let the program that provides health care for 9 million children expire. Call Congress &amp; tell them to reauthorize CHIP now: 202-224-3121 https://t.co/jk1F

In [46]:
class TweetMiner(object):    
    def __init__(self, api, result_limit = 20):        
        self.api = api        
        self.result_limit = result_limit
    def mine_user_tweets(self, user="HillaryClinton", mine_retweets=False, max_pages=20):
        data           =  []
        last_tweet_id  =  False
        page           =  1
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1, include_rts=mine_retweets)
                statuses = [ _.AsDict() for _ in statuses]
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, include_rts=mine_retweets)
                statuses = [_.AsDict() for _ in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                last_tweet_id = item['id']
                data.append(mined)
            page += 1
        return data
print("Done")

Done


In [47]:
miner = TweetMiner(api, result_limit=5000)
hillary = miner.mine_user_tweets(user="HillaryClinton", max_pages=18)
# donald = miner.mine_user_tweets(user="realDonaldTrump", max_pages=18)
modi = miner.mine_user_tweets(user="narendramodi", max_pages=18)
kimk = miner.mine_user_tweets(user="KimKardashian", max_pages=18)
kohli = miner.mine_user_tweets(user="imVkohli", max_pages=18)
print("Done")

Done


In [48]:
for x in range(5):
    print(kimk[x]['text'])
    print('---')

I know I really wanted to!!!! https://t.co/Lfoo8gCzvC
---
His cast is finally off but he’s in a brace now https://t.co/QSgAlK3DD0
---
Hi!!!! https://t.co/JyLLZKqkSu
---
No shade throwing, I don’t do subs. I want everyone to win. I just had a good caption LOL https://t.co/yf1CqtUXNH
---
Just added new items to my #KardashianKloset https://t.co/SUNF70tqmw
---


In [49]:
for x in range(5):
    print(modi[x]['text'])
    print('--')

Addressing a public meeting in Dehradun. https://t.co/i6nXAdiafu
--
Saddened by the passing away of Shri K. Rosaiah Garu. I recall my interactions with him when we both served as Chief Ministers and later when he was Tamil Nadu Governor. His contributions to public service will be remembered. Condolences to his family and supporters. Om Shanti. https://t.co/zTWyh3C8u1
--
Greetings on Navy Day. We are proud of the exemplary contributions of the Indian navy. Our navy is widely respected for its professionalism and outstanding courage. Our navy personnel have always been at the forefront of mitigating crisis situations like natural disasters. https://t.co/Cc4XgbMYuz
--
देवभूमि उत्तराखंड की विकास यात्रा में कल एक स्वर्णिम अध्याय जुड़ेगा। देहरादून में दोपहर 1 बजे मुझे 18 हजार करोड़ रुपये की कई परियोजनाओं के उद्घाटन और शिलान्यास का सौभाग्य प्राप्त होगा। इनमें दिल्ली-देहरादून आर्थिक गलियारा समेत कई प्रोजेक्ट शामिल हैं।
https://t.co/nGzlXpGVGN
--
GIFT City in Gujarat represents India’s vibranc

In [50]:
hillary_df=pd.DataFrame(hillary)
# trump_df=pd.DataFrame(donald)
modi_df=pd.DataFrame(modi)
kohli_df=pd.DataFrame(kohli)
kimk_df=pd.DataFrame(kimk)

In [51]:
print(hillary_df.shape)
# print(trump_df.shape)
print(modi_df.shape)
print(kohli_df.shape)
print(kimk_df.shape)

(2541, 6)
(3215, 6)
(2399, 6)
(2502, 6)


In [52]:
tweets = pd.concat([hillary_df, modi_df, kohli_df, kimk_df], axis=0)
tweets.shape

(10657, 6)

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')
summaries = "".join(kohli_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)
Counter(ngrams_summaries).most_common(20)

[('god bless', 47),
 ('happy birthday', 40),
 ('jai hind', 37),
 ('anushkasharma thank', 34),
 ('staywrogn https', 30),
 ('rcbtweets https', 29),
 ('anushkasharma thanks', 28),
 ('great day', 24),
 ('year ahead', 24),
 ('looking forward', 20),
 ('hind https', 19),
 ('thank wishes', 19),
 ('jai hind https', 19),
 ('anushkasharma https', 16),
 ('good health', 16),
 ('ad https', 15),
 ('day https', 15),
 ('new year', 15),
 ('playbold rcbtweets', 14),
 ('great year', 14)]

In [54]:
from textacy.preprocess import preprocess_text
tweet_text = tweets['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

In [55]:
print(clean_text[1:8])

['thank you for sharing your story billie jean url', 'today is a good day to donate to planned parenthood and their action fund url url', 'i m with staceyabrams are you url', 'looking forward to celebrating these incredible women with all of you url', 'could have been based on anyone url', 'we cannot and will not leave our granddaughters a world in which they have fewer rights and opportunities than we did', 'in one week join me benaffleck ericholder and traceeellisross to talk about an urgent threat to our democracy partisan gerrymandering and what we can about it url url']


In [56]:
temp={'HillaryClinton':1,'realDonaldTrump':2,'narendramodi':3,'KimKardashian':4,'imVkohli':5}
y = tweets['handle'].map(lambda x: temp[x]).values
print(max(pd.Series(y).value_counts(normalize=True)))

0.3016796471802571


In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
tfv = TfidfVectorizer(ngram_range=(2,4), max_features=2500)
X = tfv.fit_transform(clean_text).todense()
print(X.shape)

(10657, 2500)


In [ ]:
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression()
params = {'penalty': ['l1', 'l2'], 'C':np.logspace(-5,0,100)}
gs = GridSearchCV(lr, param_grid=params, cv=10, verbose=1)
gs.fit(X, y)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/advithinair/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/advithinair/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/advithinair/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/advithinair/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "

In [66]:
print(gs.best_params_)
print(gs.best_score_)

{'C': 1.0, 'penalty': 'l2'}
0.736323543211035


In [88]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(LogisticRegression(), X, y, cv=10)
print(accuracies.mean())

/Users/advithinair/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/advithinair/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/advithinair/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/advithinair/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/advithinair/anaconda3/lib/python3.7/site-pa

0.7363330998861379


/Users/advithinair/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [89]:
estimator = LogisticRegression(penalty='l2',C=1.0)
estimator.fit(X,y)
source_test = [
    "The presidency doesn’t change who you are—it reveals who you are. And we’ve seen all we need to of Donald Trump.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!",
    "On the banks of the Ganga, prayed for India’s peace and prosperity. Glimpses from the magnificent Ganga Aarti in Kashi",
    "Hey guys, I’m super excited to announce that Mobile Premier League is now the Number 1 Gaming App in India 🇮🇳🙏 Ab har gully se hero niklega jab saara India MPL khelega 👊👊👊 Download the @plaympl app today, play some of India's most popular games and win big everyday!",
    "Sooooo Miss North dressed herself and thought she was ready for the day until I explained she couldn’t wear my boots."
]
Xtest = tfv.transform(source_test)
pd.DataFrame(estimator.predict_proba(Xtest), columns=["Proba_Hillary","Proba_Modi", "Proba_Kim", "Proba_Kohli"])

/Users/advithinair/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/advithinair/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


,Proba_Hillary,Proba_Modi,Proba_Kim,Proba_Kohli
0,0.635843,0.025484,0.238063,0.100611
1,0.178216,0.457166,0.246741,0.117877
2,0.062191,0.735287,0.039548,0.162974
3,0.044142,0.231994,0.267905,0.455959
4,0.230130,0.076317,0.154952,0.538600


In [90]:
estimator.predict_proba(X)

array([[0.78337462, 0.06575565, 0.10097414, 0.04989559],
       [0.35572289, 0.07449911, 0.37191302, 0.19786499],
       [0.48051701, 0.07462299, 0.21498187, 0.22987813],
       ...,
       [0.14859968, 0.25398245, 0.35057088, 0.24684699],
       [0.05265889, 0.04786003, 0.75872478, 0.1407563 ],
       [0.14859968, 0.25398245, 0.35057088, 0.24684699]])

In [91]:
Probas_x = pd.DataFrame(estimator.predict_proba(X), columns= ["Proba_Hillary", "Proba_Modi", "Proba_Kim", "Proba_Kohli"])
joined_x = pd.merge(tweets, Probas_x, left_index=True, right_index=True)
joined_x

,tweet_id,handle,retweet_count,text,mined_at,created_at,Proba_Hillary,Proba_Modi,Proba_Kim,Proba_Kohli
0,1466486144443506698,HillaryClinton,1150,Trump's effort to subvert our democracy is ong...,2021-12-04 14:43:03.993041,Thu Dec 02 19:15:17 +0000 2021,0.783375,0.065756,0.100974,0.049896
0,1467050785032736768,narendramodi,1166,Addressing a public meeting in Dehradun. https...,2021-12-04 14:43:18.252132,Sat Dec 04 08:38:58 +0000 2021,0.783375,0.065756,0.100974,0.049896
0,1466398857651101697,imVkohli,10480,Back to Test cricket ❤️ https://t.co/IgjfuySCWK,2021-12-04 14:43:45.491414,Thu Dec 02 13:28:26 +0000 2021,0.783375,0.065756,0.100974,0.049896
0,1466860886899892230,KimKardashian,277,I know I really wanted to!!!! https://t.co/Lfo...,2021-12-04 14:43:31.838388,Fri Dec 03 20:04:23 +0000 2021,0.783375,0.065756,0.100974,0.049896
1,1466479975599640580,HillaryClinton,835,"Thank you for sharing your story, Billie Jean....",2021-12-04 14:43:03.993051,Thu Dec 02 18:50:46 +0000 2021,0.355723,0.074499,0.371913,0.197865
...,...,...,...,...,...,...,...,...,...,...
3210,1322488276738056192,narendramodi,1975,"Be it in the civil services or beyond, energy,...",2021-12-04 14:43:30.712338,Sat Oct 31 10:39:10 +0000 2020,0.038811,0.703142,0.077429,0.180618
3211,1322488188653457408,narendramodi,2410,I urged the young civil servants to make decis...,2021-12-04 14:43:30.712339,Sat Oct 31 10:38:49 +0000 2020,0.042824,0.520609,0.033742,0.402825
3212,1322488040615497729,narendramodi,2097,Today’s Aarambh session was held with young of...,2021-12-04 14:43:30.712341,Sat Oct 31 10:38:14 +0000 2020,0.034085,0.812470,0.059845,0.093600
3213,1322456736025006082,narendramodi,2757,हमें यह हमेशा याद रखना है कि हम सभी के लिए सर्...,2021-12-04 14:43:30.712342,Sat Oct 31 08:33:50 +0000 2020,0.101604,0.805025,0.033670,0.059701


In [101]:
import tweepy
import datetime
import xlsxwriter

consumer_key ='aTSujD4oxgvgpgeHrwGOJSJsU'
consumer_secret ='zWCl0ALnLgxeXMHssVkOJBWtaCY2wRULmuceNnlB8gV8ONjOGg'
access_key ='1121454937928388609-lMwUhwTkIWdi5Ea9uaF30XyXhwo8Ne'
access_secret ='a93kDTvNFb25UEuz8jLxuXNWPTsNjT5bHCrrbCY5pjIeV'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

api = tweepy.API(auth)
print("Done")

Done


In [102]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
username="HillaryClinton"

auth.set_access_token(access_key, access_secret) 
from textacy import preprocess_text
api = tweepy.API(auth) 

number_of_tweets=20
tweets = api.user_timeline(screen_name=username) 

tmp=[]
tweets_for_csv = [tweet.text for tweet in tweets]  
for j in tweets_for_csv:
    tmp.append(j)  
tmp=[preprocess_text(x, fix_unicode=True, lowercase=True, no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True, no_punct=True, no_accents=True) for x in tmp]

Xtest = tfv.transform(tmp[1:])
pd.DataFrame(estimator.predict_proba(Xtest), columns=["Proba_Hillary", "Proba_Modi", "Proba_Kim", "Proba_Kohli"])

,Proba_Hillary,Proba_Modi,Proba_Kim,Proba_Kohli
0,0.355723,0.074499,0.371913,0.197865
1,0.480517,0.074623,0.214982,0.229878
2,0.242342,0.061198,0.442769,0.253691
3,0.148600,0.253982,0.350571,0.246847
4,0.289988,0.066918,0.177166,0.465928
5,0.268029,0.273208,0.103951,0.354812
6,0.237710,0.429059,0.219561,0.113671
7,0.576033,0.248258,0.118941,0.056768
8,0.780888,0.104869,0.068084,0.046159
9,0.148600,0.253982,0.350571,0.246847


In [ ]:
tweets

In [ ]:
username = "HillaryClinton"
startDate = datetime.datetime(2014, 6, 1, 0, 0, 0)
endDate = datetime.datetime(2015, 1, 1, 0, 0, 0)
tweets = []
tmpTweets = api.user_timeline(username)
for tweet in tmpTweets:
    if tweet.created_at < endDate and tweet.created_at > startDate:
        tweets.append(tweet)
while (tmpTweets[-1].created_at > startDate):
    #print("Last Tweet @", tmpTweets[-1].created_at, " - fetching some more")
    tmpTweets = api.user_timeline(username, max_id = tmpTweets[-1].id)
    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)
workbook = xlsxwriter.Workbook(username + ".xlsx")
worksheet = workbook.add_worksheet()
row = 0
for tweet in tweets:
    worksheet.write_string(row, 0, str(tweet.id))
    worksheet.write_string(row, 1, str(tweet.created_at))
    worksheet.write(row, 2, tweet.text)
    worksheet.write_string(row, 3, str(tweet.in_reply_to_status_id))
    row += 1
workbook.close()